**LU Decomposition - With out pivoting**

Downloading matrix

In [1]:
import requests

matrix_id = "HB/bcsstk16"  # choose one of the IDs above
url = f"https://sparse.tamu.edu/mat/{matrix_id}.mat"
dest_filename = matrix_id.split("/")[-1] + ".mat"

# Download the file using requests
response = requests.get(url, stream=True)
response.raise_for_status()

with open(dest_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Downloaded {dest_filename}")

Downloaded bcsstk16.mat


Loading Matrix

In [2]:
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csr_matrix

# Load the .mat file
data = loadmat("bcsstk16.mat")

# Access the matrix stored in the 'Problem' struct
A = data['Problem'][0, 0]['A']  # This is typically a sparse matrix already

# If A is not in CSR format, convert it
A_sparse = csr_matrix(A)

# Load RHS vector if available
if 'b' in data['Problem'][0, 0].dtype.names:
    b = data['Problem'][0, 0]['b'].flatten()
else:
    b = np.random.rand(A_sparse.shape[0])  # Generate dummy RHS

# Inspect matrix
print("Shape:", A_sparse.shape)
print("Non-zeros:", A_sparse.nnz)

Shape: (4884, 4884)
Non-zeros: 290378


1. LU decomposition - With out pivoting

    Matrix Requirement: Square, non-singular

In [3]:
import numpy as np
from scipy.sparse import csr_matrix

def lu_no_pivot(A):
    # Convert sparse to dense
    U = np.array(A.todense(), dtype=float)
    n = U.shape[0]
    L = np.eye(n)

    # Gaussian Elimination Loop
    for i in range(n):
        if U[i, i] == 0:
            raise ZeroDivisionError(f"Zero pivot encountered at row {i}")
        # Eliminate entries below pivot
        for j in range(i+1, n):
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j, i:] = U[j, i:] - factor * U[i, i:]
    return L, U

# usage
L, U = lu_no_pivot(A_sparse)
print("L matrix:\n", L)
print("U matrix:\n", U)

# Verify correctness
A_dense = np.array(A_sparse.todense(), dtype=float)
print("Residual norm ||A - L@U||:", np.linalg.norm(A_dense - L @ U))

L matrix:
 [[ 1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.0933838   1.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  1.          0.
   0.        ]
 [ 0.          0.          0.         ... -0.93822666  1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   1.        ]]
U matrix:
 [[ 2.85559875e+08  2.66666667e+07  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.43067720e+08  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.83914177e+07
  -3.60198516e+07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   7.05117186e+07  0.00000000e+00]
 [ 0